In [ ]:
import json
import pandas as pd
import numpy as np

# Load datasets
path = 'data/processed/'

with open(path+"preprocessed_train.json", "r") as f:
    train_claims = json.load(f)

train_df = pd.DataFrame(train_claims).transpose()

with open(path+"preprocessed_dev.json", "r") as f:
    dev_claims = json.load(f)
dev_df = pd.DataFrame(dev_claims).transpose()

with open(path+"preprocessed_evidence.json", "r") as f:
    evidence = json.load(f)
evidence_df = pd.DataFrame(evidence)

In [ ]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm

hard_negatives = {}
positives = {}
top_k = 100

bm25 = BM25Okapi(evidence_df['value'].tolist())

for claim_id, row in tqdm(train_df.iterrows()):
    claim_text = row['claim_text']
    
    # Get top k from bm25
    scores = bm25.get_scores(claim_text)
    top_k_scores = (-scores).argsort()[:top_k]

    hard_negatives[claim_id] = []
    positives[claim_id] = row['evidences'] # positives are gold standard evidences

    # Save hard negatives
    for idx in top_k_scores:
        if evidence_df['key'].iloc[idx] not in row['evidences']:
            hard_negatives[claim_id].append(evidence_df['key'].iloc[idx])

1228it [41:09,  2.01s/it]


In [42]:
hard_negatived_df = pd.DataFrame.from_dict(positives, orient='index')

In [ ]:
import os

# Save hard negatives and positives

path = "data/training_data/"
if not os.path.exists(path):
    os.makedirs(path)
    print(f"Created directory {path}")

with open(path+"positives.json", "w") as f:
    f.write(json.dumps(positives, indent=4))

with open(path+"hard_negatives.json", "w") as f:
    f.write(json.dumps(hard_negatives, indent=4))